# ICマスタデータの前処理

## やっていること
- ICネットワークから各ICのdegreeを取り出し、ICマスタに付与
- ICマスタ-トラカンの区間名称対応表から、トラカンにおけるIC名称をICマスタに付与

## Input
- ICネットワークデータ: `Input_processed_data/road_master/220303-doronet_ic.csv`
- ICマスタデータ: `Input_processed_data/road_master/220303-ic.csv`
- ICマスタ-トラカン区間名称対応表: `Input_processed_data/road_master/ic_name_relationship.csv`

## Output
- 前処理済みICマスタデータ: `Input_processed_data/road_master/ic_preprocessed.csv`

## 前提条件
- ICマスタ-トラカン区間名称対応表に含まれる"JCT", "PA", "SA", "SIC", "IC"などの半角文字は全角文字に直しておく
- ICマスタ-トラカン区間名称対応表の対象範囲に注意
    - 2023/9/9現在: 関越道, 東北道, 館山道

In [1]:
import numpy as np
import pandas as pd

In [2]:
# data directory
PROCESSED_DATA_DIR = '../../Input_processed_data'
ORI_DATA_DIR = '../../Input_original_data'

# ICマスタ, ICネットワーク CSV
IC_CSV = f'{PROCESSED_DATA_DIR}/road_master/220303-ic.csv'
IC_NET_CSV = f'{PROCESSED_DATA_DIR}/road_master/220303-doronet_ic.csv'
IC_REL_CSV = f'{PROCESSED_DATA_DIR}/road_master/ic_name_relationship.csv'

## degree情報の付与

### ICネットワークデータからdegree情報を取り出す

In [3]:
# ICネットワークデータを読み込む
df_icnet = pd.read_csv(IC_NET_CSV)
df_icnet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5624 entries, 0 to 5623
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   rosen_code    5624 non-null   int64  
 1   road_code     5624 non-null   object 
 2   road_name     5624 non-null   object 
 3   direction     5624 non-null   int64  
 4   start_code    5624 non-null   object 
 5   end_code      5624 non-null   object 
 6   billing_No    5624 non-null   int64  
 7   start_name    5624 non-null   object 
 8   end_name      5624 non-null   object 
 9   start_lat     5588 non-null   float64
 10  start_lng     5588 non-null   float64
 11  end_lat       5588 non-null   float64
 12  end_lng       5588 non-null   float64
 13  distance      5624 non-null   float64
 14  start_degree  5624 non-null   int64  
 15  end_degree    5624 non-null   int64  
dtypes: float64(5), int64(5), object(6)
memory usage: 703.1+ KB


In [4]:
df_icnet.head()

,rosen_code,road_code,road_name,direction,start_code,end_code,billing_No,start_name,end_name,start_lat,start_lng,end_lat,end_lng,distance,start_degree,end_degree
0,109,1010,【E1】東名高速道路,0,1010001,1010004,1,東京,東京本線,35.62455,139.626661,35.59110,139.576578,5.86,4,4
1,109,1010,【E1】東名高速道路,0,1010004,1010006,6,東京本線,東名川崎,35.59110,139.576578,35.58415,139.573103,0.83,4,4
2,109,1010,【E1】東名高速道路,0,1010006,1010016,11,東名川崎,横浜青葉,35.58415,139.573103,35.54295,139.540811,5.43,4,4
3,109,1010,【E1】東名高速道路,0,1010016,1010018,16,横浜青葉,横浜青葉ＪＣＴ,35.54295,139.540811,35.54295,139.540811,0.00,4,6
4,109,1010,【E1】東名高速道路,0,1010018,1010011,21,横浜青葉ＪＣＴ,港北ＰＡ,35.54295,139.540811,35.53066,139.533192,1.53,6,4


In [5]:
df_icnet.tail()

,rosen_code,road_code,road_name,direction,start_code,end_code,billing_No,start_name,end_name,start_lat,start_lng,end_lat,end_lng,distance,start_degree,end_degree
5619,986,9134,目白通り,1,1800001,9134010,6,練馬,谷原交差点,35.75133,139.610825,35.74587,139.621708,1.16,6,2
5620,409,X409,国道９号,0,9020080,9020060,1,はわい,大栄東伯,35.49359,133.875875,35.49680,133.720950,14.03,4,4
5621,409,X409,国道９号,1,9020060,9020080,1,大栄東伯,はわい,35.49680,133.720950,35.49359,133.875875,14.03,4,4
5622,987,X507,国道２０号,0,9132020,1100006,1,高井戸１丁目,調布,35.66872,139.616633,35.65678,139.538814,7.16,2,6
5623,987,X507,国道２０号,1,1100006,9132020,1,調布,高井戸１丁目,35.65678,139.538814,35.66872,139.616633,7.16,6,2


In [6]:
df_start = df_icnet.loc[:, ['start_code', 'start_name', 'start_degree']]
df_end = df_icnet.loc[:, ['end_code', 'end_name', 'end_degree']]

In [7]:
print(df_start.shape)
df_start.head()

(5624, 3)


,start_code,start_name,start_degree
0,1010001,東京,4
1,1010004,東京本線,4
2,1010006,東名川崎,4
3,1010016,横浜青葉,4
4,1010018,横浜青葉ＪＣＴ,6


In [8]:
print(df_end.shape)
df_end.head()

(5624, 3)


,end_code,end_name,end_degree
0,1010004,東京本線,4
1,1010006,東名川崎,4
2,1010016,横浜青葉,4
3,1010018,横浜青葉ＪＣＴ,6
4,1010011,港北ＰＡ,4


In [9]:
df_tmp = df_start.merge(
    df_end, 
    left_on=['start_code', 'start_name'], 
    right_on=['end_code', 'end_name'], 
    how='outer'
)
df_tmp

,start_code,start_name,start_degree,end_code,end_name,end_degree
0,1010001,東京,4,1010001,東京,4
1,1010001,東京,4,1010001,東京,4
2,1010001,東京,4,1010001,東京,4
3,1010001,東京,4,1010001,東京,4
4,1010004,東京本線,4,1010004,東京本線,4
...,...,...,...,...,...,...
12335,9131040,上野吉無田,4,9131040,上野吉無田,4
12336,9131040,上野吉無田,4,9131040,上野吉無田,4
12337,9131050,山都中島西,2,9131050,山都中島西,2
12338,9134010,谷原交差点,2,9134010,谷原交差点,2


In [10]:
# 各ICのstart_degreeとend_degreeが一致することを確認
df_tmp.start_degree.equals(df_tmp.end_degree)

True

In [11]:
df_tmp.drop_duplicates()

,start_code,start_name,start_degree,end_code,end_name,end_degree
0,1010001,東京,4,1010001,東京,4
4,1010004,東京本線,4,1010004,東京本線,4
8,1010006,東名川崎,4,1010006,東名川崎,4
12,1010016,横浜青葉,4,1010016,横浜青葉,4
16,1010018,横浜青葉ＪＣＴ,6,1010018,横浜青葉ＪＣＴ,6
...,...,...,...,...,...,...
12329,9131030,小池高山,4,9131030,小池高山,4
12333,9131040,上野吉無田,4,9131040,上野吉無田,4
12337,9131050,山都中島西,2,9131050,山都中島西,2
12338,9134010,谷原交差点,2,9134010,谷原交差点,2


In [12]:
# ICとdegreeの対応表を作成
df_degree = df_tmp.drop_duplicates().loc[:, ['start_code', 'start_name', 'start_degree']]
df_degree.columns = ['ic_code', 'ic_name', 'degree']
df_degree

,ic_code,ic_name,degree
0,1010001,東京,4
4,1010004,東京本線,4
8,1010006,東名川崎,4
12,1010016,横浜青葉,4
16,1010018,横浜青葉ＪＣＴ,6
...,...,...,...
12329,9131030,小池高山,4
12333,9131040,上野吉無田,4
12337,9131050,山都中島西,2
12338,9134010,谷原交差点,2


### ICマスタデータにdegree列を付与

In [13]:
# ICマスタデータにdegree列を付与
df_ic = pd.read_csv(IC_CSV)
print(df_ic.shape)
df_ic.head()

(2734, 11)


,ic_code,ic_name,ic_reading,area_code,pref_code,type,緯度,経度,lat,lng,pref_name
0,1010001,東京,とうきょう,3,13,IC,35/37/28.38,139/37/35.98,35.62455,139.626661,東京都
1,1010004,東京本線,とうきょうほんせん,3,14,IC,35/35/27.96,139/34/35.68,35.59110,139.576578,神奈川県
2,1010006,東名川崎,とうめいかわさき,3,14,IC,35/35/2.93,139/34/23.17,35.58415,139.573103,神奈川県
3,1010011,港北ＰＡ,こうほくぱーきんぐえりあ,3,14,PA,35/31/50.36,139/31/59.49,35.53066,139.533192,神奈川県
4,1010016,横浜青葉,よこはまあおば,3,14,IC,35/32/34.62,139/32/26.92,35.54295,139.540811,神奈川県


In [14]:
# ICマスタデータにdegree列を付与
df_ic = df_ic.merge(df_degree, on=['ic_code', 'ic_name'], how='left')
print(df_ic.shape)
df_ic.head()

(2734, 12)


,ic_code,ic_name,ic_reading,area_code,pref_code,type,緯度,経度,lat,lng,pref_name,degree
0,1010001,東京,とうきょう,3,13,IC,35/37/28.38,139/37/35.98,35.62455,139.626661,東京都,4.0
1,1010004,東京本線,とうきょうほんせん,3,14,IC,35/35/27.96,139/34/35.68,35.59110,139.576578,神奈川県,4.0
2,1010006,東名川崎,とうめいかわさき,3,14,IC,35/35/2.93,139/34/23.17,35.58415,139.573103,神奈川県,4.0
3,1010011,港北ＰＡ,こうほくぱーきんぐえりあ,3,14,PA,35/31/50.36,139/31/59.49,35.53066,139.533192,神奈川県,4.0
4,1010016,横浜青葉,よこはまあおば,3,14,IC,35/32/34.62,139/32/26.92,35.54295,139.540811,神奈川県,4.0


In [15]:
print(df_ic.degree.unique())
print(df_ic.degree.value_counts())

[ 4.  6.  8.  2. nan]
4.0    2233
6.0     213
2.0     195
8.0      68
Name: degree, dtype: int64


In [16]:
df_ic.loc[df_ic.degree.isna()]

,ic_code,ic_name,ic_reading,area_code,pref_code,type,緯度,経度,lat,lng,pref_name,degree
453,1062041,いよ小松,いよこまつ,9,38,IC,33/52/55.63,133/5/24.64,33.88212,133.090178,愛媛県,NaN
466,1068020,徳島津田,とくしまつだ,9,36,IC,34/2/41.41,134/35/17.32,34.04484,134.588144,徳島県,NaN
1531,201E001,深川西本線,ふかがわにしほんせん,1,1,IC,43/40/01.15,142/01/42.64,43.66699,142.028511,北海道,NaN
1532,201E003,深川西,ふかがわにし,1,1,IC,43/42/16.16,142/00/28.83,43.70449,142.008008,北海道,NaN
1533,201E010,秩父別,ちっぷべつ,1,1,IC,43/45/42.48,141/58/59.48,43.76180,141.983189,北海道,NaN
1534,201E020,沼田(深川留萌道),ぬまた,1,1,IC,43/47/48.85,141/54/33.85,43.79690,141.909403,北海道,NaN
1535,201E030,北竜ひまわり,ほくりゅうひまわり,1,1,IC,43/49/22.65,141/50/18.65,43.82296,141.838514,北海道,NaN
1536,201E040,留萌幌糠,るもいほろぬか,1,1,IC,43/50/56.15,141/45/42.15,43.84893,141.761708,北海道,NaN
1537,201E050,留萌大和田,るもいおおわだ,1,1,IC,43/54/33.26,141/41/40.15,43.90924,141.694486,北海道,NaN
1538,201E060,留萌,るもい,1,1,IC,43/56/11.73,141/40/23.40,43.93659,141.673167,北海道,NaN


## トラカンIC名称の付与

In [3]:
df_icrel = pd.read_csv(IC_REL_CSV)
print(df_icrel.shape)
df_icrel.head()

(200, 5)


,ic_code_traffic,ic_name_traffic,KP,ic_code,ic_name
0,1800010,練馬,0.0,1800001,練馬
1,1800020,大泉ＪＣＴ,0.8,1110210,大泉ＪＣＴ
2,1800030,所沢,9.4,1800006,所沢
3,1800035,三芳ＰＡスマートＩＣ,14.0,1800011,三芳ＰＡ
4,1800040,川越,21.2,1800016,川越


In [4]:
df_icrel.tail()

,ic_code_traffic,ic_name_traffic,KP,ic_code,ic_name
195,1040510,大鰐弘前,643.656,1040431,大鰐弘前
196,1040520,黒石,653.565,1040441,黒石
197,1040530,浪岡,667.579,1040446,浪岡
198,1040535,青森ＪＣＴ,677.086,1040451,青森ＪＣＴ
199,1040540,青森,679.600,1040456,青森


In [19]:
df_ic = df_ic.merge(df_icrel, on=['ic_code', 'ic_name'], how='left')
print(df_ic.shape)
df_ic.head()

(2734, 15)


,ic_code,ic_name,ic_reading,area_code,pref_code,type,緯度,経度,lat,lng,pref_name,degree,ic_code_traffic,ic_name_traffic,KP
0,1010001,東京,とうきょう,3,13,IC,35/37/28.38,139/37/35.98,35.62455,139.626661,東京都,4.0,NaN,NaN,NaN
1,1010004,東京本線,とうきょうほんせん,3,14,IC,35/35/27.96,139/34/35.68,35.59110,139.576578,神奈川県,4.0,NaN,NaN,NaN
2,1010006,東名川崎,とうめいかわさき,3,14,IC,35/35/2.93,139/34/23.17,35.58415,139.573103,神奈川県,4.0,NaN,NaN,NaN
3,1010011,港北ＰＡ,こうほくぱーきんぐえりあ,3,14,PA,35/31/50.36,139/31/59.49,35.53066,139.533192,神奈川県,4.0,NaN,NaN,NaN
4,1010016,横浜青葉,よこはまあおば,3,14,IC,35/32/34.62,139/32/26.92,35.54295,139.540811,神奈川県,4.0,NaN,NaN,NaN


In [20]:
df_ic.loc[df_ic.ic_name_traffic.notna()]

,ic_code,ic_name,ic_reading,area_code,pref_code,type,緯度,経度,lat,lng,pref_name,degree,ic_code_traffic,ic_name_traffic,KP
253,1040001,川口ＪＣＴ,かわぐちじゃんくしょん,3,11,JCT,35/50/59.22,139/44/13.74,35.84978,139.737150,埼玉県,8.0,1040010,川口ＪＣＴ,0.0
255,1040013,浦和本線,うらわほんせん,3,11,IC,35/53/24.42,139/43/25.31,35.89012,139.723697,埼玉県,4.0,1040020,浦和,4.8
257,1040016,岩槻,いわつき,3,11,IC,35/56/4.92,139/41/26.70,35.93470,139.690750,埼玉県,4.0,1040030,岩槻,10.5
259,1040020,蓮田スマート,はすだすまーと,3,11,IC,35/59/49.19,139/40/14.44,35.99700,139.670678,埼玉県,4.0,1040032,蓮田ＳＡスマートＩＣ,18.2
261,1040023,久喜白岡ＪＣＴ,くきしらおかじゃんくしょん,3,11,JCT,36/2/44.36,139/39/40.34,36.04566,139.661206,埼玉県,8.0,1040035,久喜白岡ＪＣＴ,24.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,213A046,武石,たけいし,3,12,IC,35/39/59.61,140/4/2.15,35.66656,140.067264,千葉県,4.0,213A070,武石,17.9
1606,213A056,穴川,あながわ,3,12,IC,35/38/22.79,140/6/56.35,35.63966,140.115653,千葉県,4.0,213A090,穴川,23.8
1607,213A061,貝塚(京葉道路),かいづか,3,12,IC,35/37/13.97,140/8/45.47,35.62055,140.145964,千葉県,4.0,213A100,貝塚,27.4
1608,213A066,松ヶ丘,まつがおか,3,12,IC,35/35/37.58,140/9/5.67,35.59377,140.151575,千葉県,4.0,213A120,松ヶ丘,31.1


In [21]:
# IC名称対応表の対象範囲は関越道・東北道・館山道に限られている
df_ic.ic_name_traffic.unique()

array([nan, '川口ＪＣＴ', '浦和', '岩槻', '蓮田ＳＡスマートＩＣ', '久喜白岡ＪＣＴ', '久喜', '加須',
       '羽生', '館林', '佐野藤岡', '佐野ＳＡスマートＩＣ', '岩舟ＪＣＴ', '栃木', '栃木都賀ＪＣＴ', '鹿沼',
       '宇都宮', '上河内ＳＡスマートＩＣ', '矢板', '矢板北スマートＩＣ', '西那須野塩原', '黒磯板室', '那須',
       '那須高原ＳＡスマートＩＣ', '白河', '白河中央ＳＩＣ', '矢吹', '鏡石ＰＡ', '須賀川', '郡山南',
       '郡山中央ＳＩＣ', '郡山', '郡山ＪＣＴ', '本宮', '二本松', '福島松川ＰＡ', '福島西', '福島ＪＣＴ',
       '福島飯坂', '桑折ＪＣＴ', '国見', '白石', '村田', '村田ＪＣＴ', '菅生ＰＡ', '仙台南', '仙台宮城',
       '泉ＰＡ', '泉', '富谷ＪＣＴ', '大和', '大衡', '三本木ＰＡ', '古川', '長者原ＳＡ', '築館',
       '若柳金成', '一関', '平泉ＳＩＣ', '平泉前沢', '奥州ＳＩＣ', '水沢', '北上金ケ崎', '北上ＪＣＴ',
       '北上江釣子', '花巻南', '花巻ＪＣＴ', '花巻', '紫波', '矢巾ＳＩＣ', '盛岡南', '盛岡',
       '滝沢中央ＳＩＣ', '滝沢', '西根', '松尾八幡平', '安代ＪＣＴ', '安代', '鹿角八幡平', '十和田',
       '小坂', '小坂ＪＣＴ', '碇ケ関', '大鰐弘前', '黒石', '浪岡', '青森ＪＣＴ', '青森', '大泉ＪＣＴ',
       '高谷ＪＣＴ', '湾岸市川', '谷津船橋', '湾岸習志野', '湾岸千葉', '宮野木ＪＣＴ', '千葉北', '四街道',
       '佐倉', '酒々井', '富里', '成田ＪＣＴ', '大栄', '佐原香取', '潮来', '鉾田', '茨城空港北',
       '茨城町ＪＣＴ', '起点', '市原', '姉崎袖ヶ浦', '木更津北', '木更津ＪＣＴ', '木更津南ＪＣＴ', '君津',
     

## 保存

In [22]:
OUTPUT_CSV = f'{PROCESSED_DATA_DIR}/road_master/ic_preprocessed.csv'

In [23]:
%time df_ic.to_csv(OUTPUT_CSV, index=False)

CPU times: user 24.7 ms, sys: 1.37 ms, total: 26.1 ms
Wall time: 25.1 ms
